In [1]:
import csv
import pandas as pd
import random
import re
import time
from tqdm import tqdm, trange

In [2]:
df = pd.read_csv('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/inputTable.csv')
df.head()

,paperA,paperB,A_firstAuthorInDegree,A_firstAuthorOutDegree,A_PaperOutDegree,abstractSim,notHasCitationPurpose,ifFirstAuthorInSameAffilaition,ifSameJournal,selfCite,citedPaperJournalTypeProportionInCitingPaperReferences,citedPaperFirstAuthorHasCiteCitingPaperFirstAuthorsPaper,citingPaperAuthorsPapersIntersectionCitedPaperAuthorsPapers,B_firstAuthorInDegree,B_firstAuthorOutDegree,B_PaperOutDegree,label_ifAbnormal
0,2138715703,2007600326,0,171,28,0.637714,0.672244,0,0,0,0.027027,0,0,6,0,0,0
1,2138715703,2147617783,0,171,28,0.569197,0.723836,0,0,0,0.054054,0,0,10,0,0,0
2,2138715703,1966687480,0,171,28,0.473437,0.770526,0,0,0,0.027027,0,0,6,0,0,0
3,2138715703,2062460839,0,171,28,0.572876,0.651287,0,0,0,0.027027,0,0,6,0,0,0
4,2138715703,2134441116,0,171,28,0.524535,0.755068,1,0,0,0.054054,1,1,22,0,0,0


In [3]:
df_t = df.loc[df['label_ifAbnormal'] == 0]
df_f = df.loc[df['label_ifAbnormal'] != 0]
'''
print(len(df_t)) 4428
print(len(df_f)) 300
print(len(df)) 4728
'''

'\nprint(len(df_t)) 4428\nprint(len(df_f)) 300\nprint(len(df)) 4728\n'

In [4]:
paperaidlist = []
paperaidlist = df_t['paperA'].unique()
paperaidliststr = [str(x) for x in paperaidlist]
#print(len(paperaidlist)) 1199

In [5]:
paperbidlist = []
paperbidlist = df_t['paperB'].unique()
paperbidliststr = [str(x) for x in paperbidlist]
#print(len(paperbidlist)) 2464

In [6]:
f2 = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/allPaperAbstracts-doc2vec-Embedding-128.txt','r')
paperid_abstractdoc2vec = []
for line in f2:
    line = line.strip().split('\t')
    paperid_abstractdoc2vec.append(line[0])
#print(len(paperid_abstractdoc2vec)) 3312
#join1 = [x for x in paperid_abstractdoc2vec if x  in paperaidlist]
#print(len(join1)) 1124

# core paper 和 true cited paper

In [7]:
#随机抽样301个，去掉没有 abstract vec的
random.seed(12)
randompaperidliststr = random.sample(paperaidliststr,301)
corepaperidliststr_invec = [x for x in randompaperidliststr if x in paperid_abstractdoc2vec]
print(len(corepaperidliststr_invec))
#corepaperidlist = [int(x) for x in corepaperidliststr]
#print(len(corepaperidlist)) 286

286


In [10]:
#找到其journal id
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/Papers.txt','r')
alljournalidOf_corepaperidliststr_invec = []
allpaperidOf_corepaperidliststr_invec = []
for line in tqdm(f) :
    line = line.strip().split('\t')
    if line[0] in corepaperidliststr_invec:
        allpaperidOf_corepaperidliststr_invec.append(line[0])
        alljournalidOf_corepaperidliststr_invec.append(line[10])

172037947it [20:06, 142553.87it/s]


In [18]:
allvalidjournalid = []
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/selectedJournals.txt','r')
for line in tqdm(f) :
    line = line.strip().split('\t')
    allvalidjournalid.append(line[0])
print(len(allvalidjournalid))


7623it [00:00, 902865.60it/s]

7623


In [20]:
#删掉没有journalid的论文
#print(len(alljournalidOf_corepaperidliststr_invec))#286
corepaperidliststr = []
for paperid,journalid in zip(allpaperidOf_corepaperidliststr_invec,alljournalidOf_corepaperidliststr_invec):
    if journalid in allvalidjournalid:
        corepaperidliststr.append(paperid)
#print(len(corepaperidliststr)) #236
corepaperidlist = [int(x) for x in corepaperidliststr]
#print(len(corepaperidlist)) ##236

286
236
236


In [56]:
#真实被引用的paper，去掉没有 abstract vec的
paperidcitedbycorepaper = []
df_citedpaper = df[df['paperA'].isin(corepaperidlist)]
#print(len(df_citedpaper)) #1141
paperidcitedbycorepaperori = df_citedpaper['paperB'].unique()
#print(len(paperidcitedbycorepaperori)) #926
paperidcitedbycorepaperoristr = [str(x) for x in paperidcitedbycorepaperori]
paperbinabstractem = [x for x in paperidcitedbycorepaperoristr if x in paperid_abstractdoc2vec]
#print(len(paperbinabstractem)) #885
paperbinabstractem_uncore = [x for x in paperbinabstractem if x not in corepaperidliststr]
#print(len(paperbinabstractem_uncore))#859

859


In [27]:

f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/Papers.txt','r')
alljournalidOf_citedpaperidliststr_invec = []
allpaperidOf_citedpaperidliststr_invec = []
for line in tqdm(f) :
    line = line.strip().split('\t')
    if line[0] in paperbinabstractem:
        allpaperidOf_citedpaperidliststr_invec.append(line[0])
        alljournalidOf_citedpaperidliststr_invec.append(line[10])
f.close()

172037947it [48:19, 59325.23it/s]


In [57]:
#删掉没有journalid的论文
print(len(alljournalidOf_citedpaperidliststr_invec))#885
citedpaperidliststr = []
for paperid,journalid in zip(allpaperidOf_citedpaperidliststr_invec,alljournalidOf_citedpaperidliststr_invec):
    if journalid in allvalidjournalid:
        citedpaperidliststr.append(paperid)
print(len(citedpaperidliststr)) #764
paperidcitedbycorepaperstr = citedpaperidliststr
print(len(paperidcitedbycorepaperstr))#764
paperidcitedbycorepaper = [int(x) for x in paperidcitedbycorepaperstr]
print(len(paperidcitedbycorepaper)) ##764

885
764
764
764


In [58]:
#删掉core paper中重复的
paperidcitedbycorepaper = sorted(set(paperidcitedbycorepaper))
#print(len(paperidcitedbycorepaper)) #764
paperidcitedbycorepaper = [x for x in paperidcitedbycorepaper if x not in corepaperidlist]
#print(len(paperidcitedbycorepaper)) #738
paperidcitedbycorepaperstr = [str(x) for x in paperidcitedbycorepaper]
#print(len(paperidcitedbycorepaperstr))# 738

764
738
738


# 需要添加的异常引用的paper id+所需要用到的所有paper id

In [65]:
#把core paper分两半，后一半加入异常引用
print(len(corepaperidliststr))
corepaperidliststr1 = corepaperidliststr[0:118]
print(len(corepaperidliststr1))  #117
corepaperidliststr2 = corepaperidliststr[118:236]
print(len(corepaperidliststr2))  #117
corepaperidlist1 = corepaperidlist[0:118]
print(len(corepaperidlist1))  #117
corepaperidlist2 = corepaperidlist[118:236]
print(len(corepaperidlist2))  #117

236
118
118
118
118


In [66]:
#两半core paper的作者
authoridofcorepaper1 = []
authoridofcorepaper2 = []
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/PaperAuthorAffiliations.txt','r')

for line in tqdm(f):
    line = line.split('\t')
    if line[0] in corepaperidliststr1:
        authoridofcorepaper1.append(line[1]) #str
    if line[0] in corepaperidliststr2:
        authoridofcorepaper2.append(line[1]) #str    
print(len(authoridofcorepaper1)) #588
print(len(authoridofcorepaper2)) #753
f.close()

455999370it [31:16, 242980.01it/s]

588
753


In [67]:
authoridofcorepaper1_sorted = sorted(set(authoridofcorepaper1))
print(len(authoridofcorepaper1_sorted)) #584
authoridofcorepaper2_sorted = sorted(set(authoridofcorepaper2))
print(len(authoridofcorepaper2_sorted)) #747

#后一半core paper的作者去掉前一半的作者
authors_notin1 = [x for x in authoridofcorepaper2_sorted if x not in authoridofcorepaper1_sorted] #str
print(len(authors_notin1)) #740

584
747
740


In [68]:
#要添加的异常引用的论文
otherpaperidofcoreauthor2 = []

f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/PaperAuthorAffiliations.txt','r')

for line in tqdm(f):
    line = line.split('\t')
    if line[1] in authors_notin1:
        otherpaperidofcoreauthor2.append(line[0]) #str
        
f.close()
print(len(otherpaperidofcoreauthor2)) #68548

455999370it [1:23:16, 91269.72it/s]

68548


In [79]:
random.seed(12)
randompaperidcitedbycorepaperstr = random.sample(paperidcitedbycorepaperstr,280)
randompaperidcitedbycorepaper = [int(x) for x in randompaperidcitedbycorepaperstr]

In [73]:
otherpaperidofcoreauthor2_0 = sorted(set(otherpaperidofcoreauthor2))
print(len(otherpaperidofcoreauthor2_0)) #56306
otherpaperidofcoreauthor2_1 = [x for x in otherpaperidofcoreauthor2_0 if x not in corepaperidlist]#int
print(len(otherpaperidofcoreauthor2_1)) #56306
otherpaperidofcoreauthor2_2 = [x for x in otherpaperidofcoreauthor2_1 if x in paperid_abstractdoc2vec]#int
print(len(otherpaperidofcoreauthor2_2)) #278
otherpaperidofcoreauthor2_3 = [x for x in otherpaperidofcoreauthor2_2 if x not in randompaperidcitedbycorepaperstr]#int
print(len(otherpaperidofcoreauthor2_3)) #254
annomouspaperidstr = []#str
print(len(annomouspaperidstr)) #254

56306
56306
278
254
0


In [74]:
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/Papers.txt','r')
alljournalidOf_annmous_paperidliststr_invec = []
allpaperidOf_annmous_paperidliststr_invec = []
for line in tqdm(f) :
    line = line.strip().split('\t')
    if line[0] in otherpaperidofcoreauthor2_3:
        alljournalidOf_annmous_paperidliststr_invec.append(line[10])
        allpaperidOf_annmous_paperidliststr_invec.append(line[0])
f.close()

172037947it [17:33, 163240.87it/s]


In [76]:
annomouspaperidstr = []#str
for journalid,paperid in zip(alljournalidOf_annmous_paperidliststr_invec ,allpaperidOf_annmous_paperidliststr_invec):
    if journalid in allvalidjournalid:
        annomouspaperidstr.append(paperid)
print(len(annomouspaperidstr)) #232
annomouspaperid = [int(x) for x in annomouspaperidstr]
print(len(annomouspaperid)) ##232

232
232


In [80]:
#所有论文id
allpaperidstr = corepaperidliststr + randompaperidcitedbycorepaperstr + annomouspaperidstr
print(len(allpaperidstr)) #748 = 236+280+232
allpaperid = corepaperidlist + randompaperidcitedbycorepaper + annomouspaperid
print(len(allpaperid))

748
748


In [83]:

with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/allpaperid.txt', 'w') as cf:
    for line in allpaperidstr:
        cf.write(line+'\n')

cf.close()

# journal和paper-journal

In [82]:
journal = alljournalidOf_annmous_paperidliststr_invec + alljournalidOf_citedpaperidliststr_invec + alljournalidOf_corepaperidliststr_invec
paper = allpaperidOf_annmous_paperidliststr_invec + allpaperidOf_citedpaperidliststr_invec + allpaperidOf_corepaperidliststr_invec

alljournalid = []
publishedpaper = []
for item1,item2 in zip(journal,paper):
    if item2 in allpaperidstr:
        alljournalid.append(item1)
        publishedpaper.append(item2)
        
print(len(alljournalid))
#alljournalid = sorted(set(alljournalid))
print(len(publishedpaper))


815
815


In [84]:
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/paperjournal.txt', 'w') as cf:
    for p,j in zip(publishedpaper,alljournalid):
        cf.write(p + ' ' + j + '\n')

cf.close()

In [85]:
alljournal = sorted(set(alljournalid))
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/alljournalid.txt', 'w') as cf:
    for line in alljournal:
        cf.write(line+'\n')

cf.close()

# author 和author-paper

In [86]:
paperid_inpaperauthor = []
authorid_inpaperauthor = []
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/PaperAuthorAffiliations.txt','r')

for line in tqdm(f):
    line = line.split('\t')
    if line[0] in allpaperidstr:
        paperid_inpaperauthor.append(line[0]) #str
        authorid_inpaperauthor.append(line[1]) #str    
print(len(paperid_inpaperauthor)) #588
print(len(authorid_inpaperauthor)) #753
f.close()

455999370it [1:36:56, 78399.38it/s]

3976
3976


In [88]:
allauthor = sorted(set(authorid_inpaperauthor))
print(len(allauthor)) #3398
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/allauthorid.txt', 'w') as cf:
    for line in allauthor:
        cf.write(line+'\n')

cf.close()

3398


In [89]:
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/paperauthor.txt', 'w') as cf:
    for p,a in zip(paperid_inpaperauthor,authorid_inpaperauthor):
        cf.write(p + ' ' + a + '\n')

cf.close()

# citing relationship

In [91]:
citinginfo = df_t.loc[df_t['paperA'].isin(corepaperidlist)]
citinginfo1 = citinginfo.loc[citinginfo['paperB'].isin(randompaperidcitedbycorepaper)]
citingpaperid = list(citinginfo1['paperA'])
citedpaperid = list(citinginfo1['paperB'])

for id1 in corepaperidliststr2:
    author = []
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/paperauthor.txt', 'r')
    for line in f:
        line = line.strip().split(' ')
        if line[0] == id1:
            author.append(line[1])
    #print(len(author))
    paper = []
    for au in author:
        
        f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/paperauthor.txt', 'r')
        for line in f:
            line = line.strip().split(' ')
            if line[1] == au:
                paper.append(line[0])
        #print(len(paper))
    paper = sorted(set(paper))
    paper = [x for x in paper if x != id1]
    for i in range(len(paper)):
        citingpaperid.append(int(id1))
        citedpaperid.append(int(paper[i]))
f.close()

In [128]:
print(len(citinginfo1))#真实引用数量
print(len(citingpaperid))#总引用数量
a = len(citinginfo1)
b = len(citingpaperid)
c = b - a
print(c)

330
504
174


In [93]:
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/citinginfo.txt', 'w') as cf:
    for ing,ed in zip(citingpaperid, citedpaperid):
        cf.write(str(ing) + ' ' + str(ed) + '\n')

cf.close()

# abstract vec matrix

In [123]:
embeddingvectors = []
x = 0
print(len(allpaperidstr))#688
for ids in allpaperidstr:
    f2 = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/allPaperAbstracts-doc2vec-Embedding-128.txt','r')
    for line in f2:
        line = line.strip().split('\t')
        if line[0] == ids:
            x+=1
            vectors = line[1].split(' ')

            embeddingvectors.append(vectors[0:129]) #str
            print(len(vectors))
            
f2.close()
print(x)#688

748
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128


In [124]:
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/matrix/abstract2vec.txt","w")
 
for line in embeddingvectors:
    for item in line:
        f.write(item + ' ')
    f.write('\n')
f.close()

# journal IF matrix

In [100]:
df = pd.read_excel(io='/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/if2017.xlsx')
Full_Title = df['Full Title'].str.lower()
IF = df['IF 2016-2017']
Full_Title = list(Full_Title)
IF = list(IF)

alljournalnames = []
print(len(alljournal)) #292
for jj in alljournal:
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/selectedJournals.txt','r')
    for line in tqdm(f) :
        line = line.strip().split('\t')
        if line[0] == jj:
            alljournalnames.append(line[1])
print(len(alljournalnames))

IF_journal = []
for name in alljournalnames:
    for title,ifs in zip(Full_Title,IF):
        if title == name:
            i = Full_Title.index(title)
            score = IF[i]
            IF_journal.append(score)
print(len(IF_journal)) #292 

292


7623it [00:00, 1125974.76it/s]
7623it [00:00, 860280.35it/s]
7623it [00:00, 1181522.46it/s]
7623it [00:00, 1012354.10it/s]
7623it [00:00, 947213.14it/s]
7623it [00:00, 1008204.19it/s]
7623it [00:00, 1041539.49it/s]
7623it [00:00, 926382.90it/s]
7623it [00:00, 1065098.08it/s]
7623it [00:00, 873894.54it/s]
7623it [00:00, 951809.34it/s]
7623it [00:00, 1095534.67it/s]
7623it [00:00, 945057.32it/s]
7623it [00:00, 1052337.80it/s]
7623it [00:00, 952631.75it/s]
7623it [00:00, 1117201.14it/s]
7623it [00:00, 933523.49it/s]
7623it [00:00, 971091.25it/s]
7623it [00:00, 1099717.25it/s]
7623it [00:00, 1007664.02it/s]
7623it [00:00, 1132676.04it/s]
7623it [00:00, 897366.81it/s]
7623it [00:00, 1077917.18it/s]
7623it [00:00, 911280.27it/s]
7623it [00:00, 988963.17it/s]
7623it [00:00, 966511.88it/s]
7623it [00:00, 941578.45it/s]
7623it [00:00, 1067944.13it/s]
7623it [00:00, 854303.94it/s]
7623it [00:00, 1040929.14it/s]
7623it [00:00, 1100625.80it/s]
7623it [00:00, 447966.76it/s]
7623it [00:00, 1156939.4

7623it [00:00, 977563.81it/s]
7623it [00:00, 808036.07it/s]
7623it [00:00, 953114.51it/s]
7623it [00:00, 968620.05it/s]
7623it [00:00, 872701.90it/s]
7623it [00:00, 1001164.18it/s]
7623it [00:00, 1025833.53it/s]
7623it [00:00, 886912.05it/s]
7623it [00:00, 966774.90it/s]
7623it [00:00, 905242.90it/s]
7623it [00:00, 1041539.49it/s]
7623it [00:00, 934888.29it/s]
7623it [00:00, 886592.34it/s]
7623it [00:00, 1041132.51it/s]
7623it [00:00, 923439.79it/s]
7623it [00:00, 961021.32it/s]
7623it [00:00, 825093.01it/s]
7623it [00:00, 869900.13it/s]
7623it [00:00, 1036407.76it/s]
7623it [00:00, 922533.89it/s]
7623it [00:00, 984456.54it/s]
7623it [00:00, 1000005.61it/s]
7623it [00:00, 837257.24it/s]
7623it [00:00, 955278.74it/s]
7623it [00:00, 1056685.15it/s]


292
292


In [102]:

f=open("/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/matrix/IFS.txt","w")
 
for line in IF_journal:
    f.write(str(line))
    f.write('\n')
f.close()

# author credit matrix

In [106]:
#author credit
df = pd.read_csv('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/inputTable.csv')
df_annomous = df.loc[df['label_ifAbnormal'] == 1]
dfa = []
dfb = []
dfa = df_annomous['paperA'].unique()
dfb = df_annomous['paperB'].unique()
creditpapers = []
creditpapers = list(dfa) + list(dfb) #int
creditpapers = sorted(set(creditpapers),key = creditpapers.index)
creditpaperstr = [str(x) for x in creditpapers]

In [107]:
annmousauthors = []
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/PaperAuthorAffiliations.txt','r')

for line in tqdm(f):
    line = line.split('\t')
    if line[0] in creditpaperstr:
        annmousauthors.append(line[1]) #str

f.close()
print(len(annmousauthors)) #2014

455999370it [36:11, 210036.11it/s]

2014


In [108]:
annmousauthorslist = sorted(set(annmousauthors))
print(len(annmousauthorslist)) #1779

1779


In [109]:
matrixcredic = [[1 for i in range(2)] for j in range(len(allauthor))]
#matrixcredic.shape
i = 0
h = 0
for i in range(len(allauthor)):
    matrixcredic[i][0] = allauthor[i]
    i += 1
for item in matrixcredic:
    if str(item[0]) in annmousauthors:
        item[1] = 0
        h += 1
credit = []
for item in matrixcredic:
    credit.append(item[1])#int
print(len(credit)) #3398
print(h) #131

3398
131


In [110]:
creditstr = [str(x) for x in credit]
f=open("/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/matrix/authorcredit.txt","w")
 
for line in creditstr:
    f.write(line)
    f.write('\n')
f.close()

# journal paper matrix

In [112]:
journalpapermatrix = [[0 for i in range(len(allpaperidstr))] for j in range(len(alljournal))]

for ids,js in zip(allpaperidstr,alljournal):
    j = allpaperidstr.index(ids)
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/allpaperid.txtpaperjournal.txt', 'r')
    for line in f:
        line = line.strip().split(' ')
        if line[0] == ids:
            i = alljournal.index(js)
            journalpapermatrix[i][j] = 1

In [113]:
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/matrix/journalpaper.txt","w")
 
for line in journalpapermatrix:
    for item in line:
        f.write(str(item)+ ' ')
    f.write('\n')
f.close()

# author author matrix

In [114]:
coauthormatrix = [[0 for i in range(len(allauthor))] for j in range(len(allauthor))]
for paperid in allpaperidstr:
    authors = []
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/paperauthor.txt', 'r')
    for line in f:
        line = line.strip().split(' ')
        if line[0] == paperid:
            authors.append(line[1])
    for author1 in authors:
        i = allauthor.index(author1)
        for author2 in authors:
            j = allauthor.index(author2)
            coauthormatrix[i][j] += 1

In [115]:
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/matrix/authorauthor.txt","w")
 
for line in coauthormatrix:
    for item in line:
        f.write(str(item)+ ' ')
    f.write('\n')
f.close()

# author paper matrix

In [116]:
print(len(citingpaperid)) #504
print(len(citedpaperid))#504
print(len(citinginfo1))#330

504
504
330


In [120]:
authorpapermatrix = [[0 for i in range(len(allpaperidstr))] for j in range(len(allauthor))]
allpaperid = [int(x) for x in allpaperid]
for a,b in zip(citingpaperid,citedpaperid):
    j = allpaperid.index(b)
    author = []
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/paperauthor.txt', 'r')
    for line in f:
        line = line.strip().split(' ')
        if int(line[0]) == a:
            author.append(line[1])
    for au in author:
        i = allauthor.index(au)
        authorpapermatrix[i][j] = 1
    
f.close()

In [121]:
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/matrix/authorpaper.txt","w")
 
for line in tqdm(authorpapermatrix):
    for item in line:
        f.write(str(item)+' ')
    f.write('\n')
f.close()

100%|█████████████████████████████████████| 3398/3398 [00:00<00:00, 3971.06it/s]


# 计算引用链接数量和合作链接数量

In [135]:
sum(map(sum,authorpapermatrix)) #引用链接数量

3246

In [136]:
sum(map(sum,coauthormatrix))#合作链接数量

41951

In [137]:
s = 0
for i in range(len(allauthor)):
    s += coauthormatrix[i][i]
    i += 1
print(s)

4729


In [138]:
h = (sum(map(sum,coauthormatrix)) - s)/2
h

18611.0